# Now it's time to work on the final group project.  
All of the pieces we've learned are integrated into this initial template to work on improving F1 measure on classification of pneumonia evidence

## A few ideas of how to improve your system:
* Improve targets.  Are there any False Negatives your system is missing?  Are there regular expressions that would help?
* Improve modifiers.  Not all modifiers typically used in practice are the modifiers starter file.  Are there some to add?  Do some existing modifiers cause problems in your processing?  They can be changed or removed.
* Improve document classification rules.  What rules work best?  What is the best "default" classification?
* Consider handling of document "sections".  Are there certain headers or subsections which are more or less likely to contain evidence?  You could modify your own "markup" function to do this or you could add Modifiers to do this in some cases

## Also before we get going, a few Pro Tips:
* Remember that pyConText files need to be tab delimited.  If you edit these files in JupyterHub, it might be difficult to see the tabs and if you press "TAB" you will actually get spaces, so try to use Copy-and-Paste
* Classification rules and modifiers are difficult.  Don't be afraid to ask for help

In [2]:
#import everything that we will need
import pyConTextNLP
from pyConTextNLP import pyConTextGraph
from pyConTextNLP.itemData import itemData
from pyConTextNLP.display.html import mark_document_with_html
import os
import os.path
# useful utilities in RadNLP as well
import radnlp
import radnlp.view as rview
from radnlp.data import classrslts
# our utilities from the class
from nlp_pneumonia_utils import Annotation
from nlp_pneumonia_utils import AnnotatedDocument
from nlp_pneumonia_utils import read_brat_annotations
from nlp_pneumonia_utils import read_doc_annotations
from nlp_pneumonia_utils import read_annotations
from nlp_pneumonia_utils import calculate_prediction_metrics
from nlp_pneumonia_utils import mark_text
from nlp_pneumonia_utils import clearPyConTextRegularExpressions
from nlp_pneumonia_utils import pneumonia_annotation_html_markup
from nlp_pneumonia_utils import mark_document_with_html
from nlp_pneumonia_utils import view_single_sentence_graph
from nlp_pneumonia_utils import markup_sentence
from nlp_pneumonia_utils import markup_context_document
from nlp_pneumonia_utils import DocumentClassifier
# packages for interaction
from ipywidgets import interact, interactive, fixed
from IPython.display import display, HTML, Image
import ipywidgets
print('Everything imported')

Everything imported


## Load our training set

In [3]:
annotated_doc_map = read_doc_annotations('data/training_v2.zip')

# let's also use a simple list of documents as well as this map
annotated_docs = list(annotated_doc_map.values())

print('Total Annotated Documents : {0}'.format(len(annotated_docs)))

Reading annotations from file : data/training_v2.zip
Opening local file : data/training_v2.zip
Total Annotated Documents : 70


## Setting up our resources
You're welcome to start new files or continue in the files we've used, but let's set up some defaults we have used in the course

In [4]:
TARGETS_FILE_PATH = 'file:///' + os.path.join(os.getcwd(), 'KB/pneumonia_targets.tsv')
MODIFIERS_FILE_PATH = 'file:///' + os.path.join(os.getcwd(),'KB/pneumonia_modifiers.tsv')
CLASSIFIER_FILE_PATH = 'KB/classifierRules.csv'

## Load our targets and resources now

In [7]:
# clear just in case files/regular expressions have been updated
clearPyConTextRegularExpressions()

targets = pyConTextNLP.itemData.instantiateFromCSVtoitemData(TARGETS_FILE_PATH)
modifiers = pyConTextNLP.itemData.instantiateFromCSVtoitemData(MODIFIERS_FILE_PATH)

print('Targets loaded : {0}'.format(len(targets)))
print('Modifiers loaded : {0}'.format(len(modifiers)))

Targets loaded : 3
Modifiers loaded : 380


## Construct our Document Classifier

In [9]:
debug_classifier = False
docClassifier = DocumentClassifier(CLASSIFIER_FILE_PATH, debug_classifier, modifiers, targets) 

## Let's attempt some predictions
* You will do a lot of iterations modifying content and then coming back here to check performane
* Remember : the prediction function passed here passes in a string (text) and returns a 0 or 1

In [10]:
print('****************')
print('Performance for Classifier :')
calculate_prediction_metrics(annotated_docs, docClassifier.predict)

****************
Performance for Classifier :

**********************************************************************
  Resource punkt not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt')
  
  Searched in:
    - '/home/jovyan/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/opt/conda/nltk_data'
    - '/opt/conda/lib/nltk_data'
    - ''
**********************************************************************



MissingCorpusError: 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://nltk.org/data.html
If this doesn't fix the problem, file an issue at https://github.com/sloria/TextBlob/issues.


## Development of your system:
* We have found the tools below for highlighting and graphing False Positives and False Negatives to be very useful.  We've provided them below in case it helps you as well

In [11]:
# NOTE : You may need to modify this color mapping if you add  Target or Modifier categories not found here
# prepare some colors for displaying any markup we might see
colors = {
    "evidence_of_pneumonia": "orange",
    "definite_negated_existence": "red",
    "probable_negated_existence": "indianred",
    "ambivalent_existence": "orange",
    "probable_existence": "forestgreen",
    "definite_existence": "green",
    "historical": "goldenrod",
    "indication": "pink",
    "acute": "golden"
}

In [12]:
# This function let's us iterate through all documents and view the markup
def view_pycontext_graph(class_results, colors):
    @interact(i=ipywidgets.IntSlider(min=0, max=len(class_results)-1))
    def _view_markup(i):
        class_result = class_results[i]
        rview.markup_to_pydot(class_result)
        display(Image("tmp.png"))
        
        report_html = mark_document_with_html(class_result.context_document, colors = colors, default_color="black")
        
        display(HTML(report_html))
        
# This function let's us iterate through all documents and view the markup
def view_annotation_markup(anno_docs, colors):
    @interact(i=ipywidgets.IntSlider(min=0, max=len(anno_docs)-1))
    def _view_markup(i):
        report_html = pneumonia_annotation_html_markup(anno_docs[i])
        
        display(HTML(report_html))

In [13]:
def list_false_negatives(gold_docs, prediction_function):
    fn_docs={}
    for doc_name, gold_doc in gold_docs.items():
        gold_label=gold_doc.positive_label;
        pred_label = prediction_function(gold_doc.text)
        if gold_label==1 and pred_label==0:
            fn_docs[doc_name]=gold_doc            
    return fn_docs  

def list_false_positives(gold_docs, prediction_function):
    fn_docs={}
    for doc_name, gold_doc in gold_docs.items():
        gold_label=gold_doc.positive_label;
        pred_label = prediction_function(gold_doc.text)
        if gold_label==0 and pred_label==1:
            fn_docs[doc_name]=gold_doc            
    return fn_docs  

In [15]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [17]:

# get our current set of false negatives and false positives if we use our simple toy classifier
# which uses targets and a simplified implementation of modifiers
current_false_negatives = list_false_negatives(annotated_doc_map, docClassifier.predict)
current_false_positives = list_false_positives(annotated_doc_map, docClassifier.predict)

# prepare each of these for visualization
fn_report_results = []
fp_report_results = []
print('Marking up False Negatives')
for anno_doc in current_false_negatives.values():
    report_context = markup_context_document(anno_doc.text, modifiers, targets)
    # package this up into a class that the RadNLP utilities can use
    results = classrslts(context_document=report_context, exam_type="Chest X-Ray", report_text=anno_doc.text, classification_result='N/A')
    fn_report_results.append(results)
    
print('Marking up False Positives')
for anno_doc in current_false_positives.values():
    report_context = markup_context_document(anno_doc.text, modifiers, targets)
    # package this up into a class that the RadNLP utilities can use
    results = classrslts(context_document=report_context, exam_type="Chest X-Ray", report_text=anno_doc.text, classification_result='N/A')
    fp_report_results.append(results)

print('Current total False Negatives : {0}'.format(len(current_false_negatives)))
print('Current total False Positives : {0}'.format(len(current_false_positives)))

Marking up False Negatives
Marking up False Positives
Current total False Negatives : 6
Current total False Positives : 3


## For False Negatives, it's most useful to see the expert span annotations for positive pneumonia evidence to see if there may be targets that should be added

In [18]:
view_pycontext_graph(fn_report_results, colors)

## For False Positives, it's most useful to see a pyConText graph since there may need to be modifiers adjusted so that targets can be properly utilized in classification

In [19]:
view_pycontext_graph(fp_report_results, colors)

# TEST SET Evaluation 
* We've been waiting for the test set.  It will not be available until the morning of the final class session.
* At that time, you can uncomment this code and make any changes to it as instructed by the class instructors:

In [ ]:
#test_doc_map = read_doc_annotations('data/training_v2.zip')

# let's also use a simple list of documents as well as this map
#test_docs = list(test_doc_map.values())

#print('Total Test Documents : {0}'.format(len(test_docs)))

# and now let's check performance on the TEST set...
#print('****************')
#print('Performance for Classifier on TEST set :')
#calculate_prediction_metrics(test_docs, docClassifier.predict)

<br/><br/>This material presented as part of the DeCART Data Science for the Health Science Summer Program at the University of Utah in 2017.<br/>
Presenters : Dr. Wendy Chapman, Jianlin Shi and Kelly Peterson